In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df

import datetime
from datetime import datetime as dt
import matplotlib.pyplot as plt

import os

from sklearn.neighbors import KNeighborsRegressor

In [2]:
print(os.getcwd())
DATA_FOLDER = '/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Data/'
RESULTS_FOLDER = '/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Results/'
CODE_FOLDER = '/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Code/'

EIA_DATA_FOLDER = DATA_FOLDER + 'EIA/'
EIA860_DATA_FOLDER = EIA_DATA_FOLDER + 'EIA-Annual-Plant-Level-Capacity-860/'
EIA923_DATA_FOLDER = EIA_DATA_FOLDER + 'EIA-Annual-Monthly-Generation-Plant-Level-923/'

FILTER_DATA_STORE_PATH = EIA_DATA_FOLDER + 'Final-Data-Filter/'

if not os.path.isdir(FILTER_DATA_STORE_PATH):
    os.mkdir(FILTER_DATA_STORE_PATH)


/Users/vbc5136/Documents/Acads/3-Fall23/RA:Thesis/_SRA-2023/_Code


#### 1. Read EIA-860 files

In [6]:
PLANT_INFO = 'Plant'
GEN_INFO = 'Generator'

In [7]:
eia860_folders = os.listdir(EIA860_DATA_FOLDER)
eia860_folders.remove('.DS_Store')
eia860_folders.sort()
eia860_folders

['eia8602011',
 'eia8602012',
 'eia8602013',
 'eia8602014',
 'eia8602015',
 'eia8602016',
 'eia8602017',
 'eia8602018',
 'eia8602019',
 'eia8602020',
 'eia8602021',
 'eia8602022']

In [8]:
required_files_dict = {}
for dir_ in eia860_folders:
    required_files_dict[dir_] = {}
    files_list = os.listdir(EIA860_DATA_FOLDER + dir_)
    print(dir_)
    required_files_list = [file for file in files_list if (
        PLANT_INFO in file or GEN_INFO in file) and ('~$' not in file)]
    plant_file = [file for file in files_list if (
        PLANT_INFO in file) and ('~$' not in file)][0]
    generator_file = [file for file in files_list if (
        GEN_INFO in file) and ('~$' not in file)][0]
    print(required_files_list)
    required_files_dict[dir_][PLANT_INFO] = plant_file
    required_files_dict[dir_][GEN_INFO] = generator_file
    print('###########################')
print(required_files_dict)

eia8602011
['Plant.xlsx', 'GeneratorY2011.xlsx']
###########################
eia8602012
['PlantY2012.xlsx', 'GeneratorY2012.xlsx']
###########################
eia8602013
['3_1_Generator_Y2013.xlsx', '2___Plant_Y2013.xlsx']
###########################
eia8602014
['2___Plant_Y2014.xlsx', '3_1_Generator_Y2014.xlsx']
###########################
eia8602015
['3_1_Generator_Y2015.xlsx', '2___Plant_Y2015.xlsx']
###########################
eia8602016
['2___Plant_Y2016.xlsx', '3_1_Generator_Y2016.xlsx']
###########################
eia8602017
['3_1_Generator_Y2017.xlsx', '2___Plant_Y2017.xlsx']
###########################
eia8602018
['2___Plant_Y2018.xlsx', '3_1_Generator_Y2018.xlsx']
###########################
eia8602019
['3_1_Generator_Y2019.xlsx', '2___Plant_Y2019.xlsx']
###########################
eia8602020
['2___Plant_Y2020.xlsx', '3_1_Generator_Y2020.xlsx']
###########################
eia8602021
['3_1_Generator_Y2021.xlsx', '2___Plant_Y2021.xlsx']
###########################
eia8602022
['

##### Read EIA-860 Plant.xlsx

In [10]:
# ONLY NEED TO RUN ONCE - DONE
raw_plant_df_dict = {}
for dir_ in required_files_dict:
    raw_plant_df = pd.read_excel(
        EIA860_DATA_FOLDER + dir_ + '/' + required_files_dict[dir_][PLANT_INFO])
    print(raw_plant_df.shape, dir_, required_files_dict[dir_][PLANT_INFO])
    raw_plant_df = raw_plant_df.rename(columns=raw_plant_df.iloc[0])
    raw_plant_df = raw_plant_df.drop(raw_plant_df.index[0])
    raw_plant_df.reset_index(drop=True)
    print(raw_plant_df.shape)

#     print(raw_plant_df.columns)
    print('#########################')
    standard_cols = raw_plant_df.columns.str.lower()
#     print(standard_cols)
    print('#########################')
    raw_plant_df.columns = standard_cols
    standard_cols = raw_plant_df.columns.str.replace(' ', '_')
#     print(standard_cols)
    print('#########################')
    raw_plant_df.columns = standard_cols
#     print(raw_plant_df.columns)

    raw_plant_df_dict[dir_] = raw_plant_df


(6856, 26) eia8602011 Plant.xlsx
(6855, 26)
#########################
#########################
#########################
(7290, 28) eia8602012 PlantY2012.xlsx
(7289, 28)
#########################
#########################
#########################
(8061, 36) eia8602013 2___Plant_Y2013.xlsx
(8060, 36)
#########################
#########################
#########################
(8521, 36) eia8602014 2___Plant_Y2014.xlsx
(8520, 36)
#########################
#########################
#########################
(8929, 36) eia8602015 2___Plant_Y2015.xlsx
(8928, 36)
#########################
#########################
#########################
(9712, 42) eia8602016 2___Plant_Y2016.xlsx
(9711, 42)
#########################
#########################
#########################
(10128, 42) eia8602017 2___Plant_Y2017.xlsx
(10127, 42)
#########################
#########################
#########################
(10981, 42) eia8602018 2___Plant_Y2018.xlsx
(10980, 42)
#########################
#######

##### Read EIA-860 Generator.xlsx

In [12]:
# ONLY NEED TO RUN ONCE - DONE
raw_generator_df_dict = {}
for dir_ in required_files_dict:
    raw_generator_df = pd.read_excel(
        EIA860_DATA_FOLDER + dir_ + '/' + required_files_dict[dir_][GEN_INFO])
    print(raw_generator_df.shape, dir_, required_files_dict[dir_][GEN_INFO])
    raw_generator_df = raw_generator_df.rename(columns=raw_generator_df.iloc[0])
    raw_generator_df = raw_generator_df.drop(raw_generator_df.index[0])
    raw_generator_df.reset_index(drop=True)
    print(raw_generator_df.shape)

#     print(raw_generator_df.columns)
    print('#########################')
    standard_cols = raw_generator_df.columns.str.lower()
#     print(standard_cols)
    print('#########################')
    raw_generator_df.columns = standard_cols
    standard_cols = raw_generator_df.columns.str.replace(' ', '_')
#     print(standard_cols)
    print('#########################')
    raw_generator_df.columns = standard_cols
#     print(raw_generator_df.columns)

    raw_generator_df_dict[dir_] = raw_generator_df


(18531, 60) eia8602011 GeneratorY2011.xlsx
(18530, 60)
#########################
#########################
#########################
(19024, 60) eia8602012 GeneratorY2012.xlsx
(19023, 60)
#########################
#########################
#########################
(19244, 71) eia8602013 3_1_Generator_Y2013.xlsx
(19243, 71)
#########################
#########################
#########################
(19746, 72) eia8602014 3_1_Generator_Y2014.xlsx
(19745, 72)
#########################
#########################
#########################
(20070, 72) eia8602015 3_1_Generator_Y2015.xlsx
(20069, 72)
#########################
#########################
#########################
(20726, 73) eia8602016 3_1_Generator_Y2016.xlsx
(20725, 73)
#########################
#########################
#########################
(21439, 73) eia8602017 3_1_Generator_Y2017.xlsx
(21438, 73)
#########################
#########################
#########################
(22120, 73) eia8602018 3_1_Generator_Y2018.x

##### Plant & Generator files common

In [13]:
plant_df_columns_list = [set(
    raw_plant_df_dict[dir_].columns) for dir_ in raw_plant_df_dict]
print(plant_df_columns_list[0], len(plant_df_columns_list), len(plant_df_columns_list[0]))


{'ferc_small_power_docket', 'primary_purpose', 'ferc_cogen', 'gridvoltage', 'name_of_water_source', 'sector_name', 'ferc_exempt_wholesale_docket', 'ownertransdist', 'utility_id', 'ownertransid', 'regulatory_status', 'state', 'nerc', 'zip', 'plant_name', 'ownerstate', 'iso_rto_code', 'plant_code', 'ferc_exempt_wholesale', 'city', 'ferc_small_power', 'street_address', 'county', 'ferc_cogen_docket', 'iso_rto', 'sector'} 12 26


In [14]:
gen_df_columns_list = [set(
    raw_generator_df_dict[dir_].columns) for dir_ in raw_generator_df_dict]
print(gen_df_columns_list[0], len(gen_df_columns_list), len(gen_df_columns_list[0]))


{'sfg_system', 'energy_source_4', 'planned_derates_month', 'other_mods', 'generator_id', 'status', 'planned_derates_net_winter_cap', 'planned_uprates_net_winter_cap', 'planned_new_primemover', 'planned_repower_month', 'supercritical', 'nameplate_capacity_mw', 'pulverized_coal', 'startup_source_4', 'energy_source_1', 'sector_name', 'subcritical', 'ownership', 'planned_derates_net_summer_cap', 'utility_id', 'operating_month', 'utility_name', 'energy_source_2', 'multiple_fuels', 'planned_energy_source_1', 'planned_repower_year', 'energy_source_5', 'startup_source_2', 'planned_uprates_net_summer_cap', 'planned_retirement_month', 'ultrasupercritical', 'state', 'operating_year', 'energy_source_6', 'planned_uprates_month', 'startup_source_1', 'plant_name', 'energy_source_3', 'deliver_power_transgrid', 'topping_bottoming', 'planned_derates_year', 'other_mod_year', 'prime_mover', 'unit_code', 'plant_code', 'duct_burners', 'summer_capacity_mw', 'fluidized_bed', 'planned_retirement_year', 'county

In [15]:
eia860_common_cols = []
eia860_union_cols = []
for plant_cols, gen_cols in zip(plant_df_columns_list, gen_df_columns_list):
    eia860_common_cols.append(gen_cols & plant_cols)
    eia860_union_cols.append(gen_cols | plant_cols)
eia860_common_cols = set.intersection(*eia860_common_cols)
eia860_union_cols = set.intersection(*eia860_union_cols)

print(eia860_common_cols, len(eia860_common_cols))
print(eia860_union_cols, len(eia860_union_cols))

{'utility_id', 'county', 'plant_name', 'sector', 'plant_code', 'sector_name', 'state'} 7
{'energy_source_4', 'generator_id', 'status', 'planned_repower_month', 'nameplate_capacity_mw', 'startup_source_4', 'energy_source_1', 'name_of_water_source', 'sector_name', 'ownership', 'utility_id', 'operating_month', 'utility_name', 'energy_source_2', 'planned_energy_source_1', 'planned_repower_year', 'energy_source_5', 'regulatory_status', 'planned_retirement_month', 'startup_source_2', 'state', 'operating_year', 'energy_source_6', 'zip', 'startup_source_1', 'plant_name', 'energy_source_3', 'prime_mover', 'plant_code', 'unit_code', 'duct_burners', 'city', 'summer_capacity_mw', 'planned_retirement_year', 'street_address', 'county', 'startup_source_3', 'winter_capacity_mw', 'sector'} 39


In [16]:
ENERGY_SOURCE = 'energy_source_1'
NAMEPLATE_CAPACITY = 'nameplate_capacity_mw'
SUMMER_CAPACITY = 'summer_capacity_mw'
WINTER_CAPACITY = 'winter_capacity_mw'
OPERATING_YEAR = 'operating_year'
OPERATING_MONTH = 'operating_month'

REQUIRED_ENERGY_SOURCES = ['SUN', 'WND', 'WAT', 'GEO']
operating_cols = [OPERATING_MONTH, OPERATING_YEAR]
capacity_cols = [NAMEPLATE_CAPACITY, SUMMER_CAPACITY, WINTER_CAPACITY]

eia860_generator_cols = [ENERGY_SOURCE] + [capacity_cols[0]] + operating_cols
print(eia860_generator_cols)

['energy_source_1', 'nameplate_capacity_mw', 'operating_month', 'operating_year']


##### Combine Plant & Generator dfs

In [17]:
plant_grouped_df_dict = {}

for key in raw_plant_df_dict:
    raw_plant_df = raw_plant_df_dict[key]
    raw_plant_df = raw_plant_df[list(eia860_common_cols)]
    raw_plant_df = raw_plant_df[pd.isna(raw_plant_df['plant_code']) == False]
    plant_grouped_df = raw_plant_df.groupby('plant_code').agg({
        'sector': pd.Series.mode,
        'plant_name': pd.Series.mode,
        'state': pd.Series.mode,
        'county': pd.Series.mode})
    plant_grouped_df.reset_index(inplace=True)
    plant_grouped_df_dict[key] = plant_grouped_df
#     plant_grouped_df.to_excel(FILTER_DATA_STORE_PATH + key + '_plant_group_df.xlsx')
    print('##########', key, ' DONE')


########## eia8602011  DONE
########## eia8602012  DONE
########## eia8602013  DONE
########## eia8602014  DONE
########## eia8602015  DONE
########## eia8602016  DONE
########## eia8602017  DONE
########## eia8602018  DONE
########## eia8602019  DONE
########## eia8602020  DONE
########## eia8602021  DONE
########## eia8602022  DONE


In [18]:
gen_grouped_df_dict = {}

for key in raw_generator_df_dict:
    raw_generator_df = raw_generator_df_dict[key]
    raw_generator_df = raw_generator_df[['plant_code'] + eia860_generator_cols]
    raw_generator_df = raw_generator_df[pd.isna(raw_generator_df['plant_code']) == False]
    raw_generator_df = raw_generator_df[raw_generator_df[ENERGY_SOURCE].isin(
        REQUIRED_ENERGY_SOURCES)]
    gen_grouped_df = raw_generator_df.groupby(
        ['plant_code', 'operating_year', 'operating_month', 'energy_source_1']).sum()
    gen_grouped_df.reset_index(inplace=True)
    gen_grouped_df_dict[key] = gen_grouped_df
#     gen_grouped_df.to_excel(FILTER_DATA_STORE_PATH + key + '_gen_group_df.xlsx')
    print('##########', key, ' DONE')


########## eia8602011  DONE
########## eia8602012  DONE
########## eia8602013  DONE
########## eia8602014  DONE
########## eia8602015  DONE
########## eia8602016  DONE
########## eia8602017  DONE
########## eia8602018  DONE
########## eia8602019  DONE
########## eia8602020  DONE
########## eia8602021  DONE
########## eia8602022  DONE


In [19]:
common_plant_grouped_list = list(plant_grouped_df_dict[eia860_folders[0]]['plant_code'])
print(len(common_plant_grouped_list))

for name in eia860_folders[1:]:
    plant_code_list = list(plant_grouped_df_dict[name]['plant_code'])
    common_plant_grouped_list = list(set(common_plant_grouped_list) & set(plant_code_list))
    print(len(common_plant_grouped_list), name)

print(len(common_plant_grouped_list), '#########')

common_gen_grouped_list = list(gen_grouped_df_dict[eia860_folders[0]]['plant_code'])
print(len(common_gen_grouped_list))

for name in eia860_folders[1:]:
    gen_code_list = list(gen_grouped_df_dict[name]['plant_code'])
    common_gen_grouped_list = list(set(common_gen_grouped_list) & set(gen_code_list))
    print(len(common_gen_grouped_list), name)

print(len(common_gen_grouped_list), '#########')
print(set(common_gen_grouped_list) == set(
    common_gen_grouped_list) & set(common_plant_grouped_list))
# common_gen_grouped_list is a proper subset of common_plant_grouped_list

6855
6781 eia8602012
6705 eia8602013
6617 eia8602014
6569 eia8602015
6517 eia8602016
6461 eia8602017
6411 eia8602018
6345 eia8602019
6280 eia8602020
6184 eia8602021
6134 eia8602022
6134 #########
3721
2384 eia8602012
2367 eia8602013
2360 eia8602014
2345 eia8602015
2332 eia8602016
2325 eia8602017
2317 eia8602018
2293 eia8602019
2263 eia8602020
2243 eia8602021
2227 eia8602022
2227 #########
True


In [20]:
final_gen_grouped_df_dict = {}
final_plant_grouped_df_dict = {}

for name in eia860_folders:
    gen_grouped_df = gen_grouped_df_dict[name]
    print(gen_grouped_df.shape)
    gen_grouped_df = gen_grouped_df[gen_grouped_df['plant_code'].isin(
        common_gen_grouped_list)]
    gen_grouped_df.reset_index(inplace=True, drop=True)
    final_gen_grouped_df_dict[name] = gen_grouped_df
#     gen_grouped_df.to_excel(FILTER_DATA_STORE_PATH + name + '_final_gen_group_df.xlsx')
    print(gen_grouped_df.shape, name)
    
    plant_grouped_df = plant_grouped_df_dict[name]
    print(plant_grouped_df.shape)
    plant_grouped_df = plant_grouped_df[plant_grouped_df['plant_code'].isin(
        common_gen_grouped_list)]
    plant_grouped_df.reset_index(inplace=True, drop=True)
    final_plant_grouped_df_dict[name] = plant_grouped_df
#     plant_grouped_df.to_excel(FILTER_DATA_STORE_PATH + name + '_final_plant_group_df.xlsx')
    print(plant_grouped_df.shape, name)
    print('######################')


(3721, 5)
(3494, 5) eia8602011
(6855, 5)
(2227, 5) eia8602011
######################
(4085, 5)
(3521, 5) eia8602012
(7289, 5)
(2227, 5) eia8602012
######################
(4401, 5)
(3530, 5) eia8602013
(8060, 5)
(2227, 5) eia8602013
######################
(4790, 5)
(3532, 5) eia8602014
(8520, 5)
(2227, 5) eia8602014
######################
(5184, 5)
(3539, 5) eia8602015
(8928, 5)
(2227, 5) eia8602015
######################
(5830, 5)
(3543, 5) eia8602016
(9711, 5)
(2227, 5) eia8602016
######################
(6396, 5)
(3542, 5) eia8602017
(10127, 5)
(2227, 5) eia8602017
######################
(6925, 5)
(3547, 5) eia8602018
(10980, 5)
(2227, 5) eia8602018
######################
(7516, 5)
(3543, 5) eia8602019
(11833, 5)
(2227, 5) eia8602019
######################
(8211, 5)
(3543, 5) eia8602020
(12636, 5)
(2227, 5) eia8602020
######################
(8913, 5)
(3545, 5) eia8602021
(13455, 5)
(2227, 5) eia8602021
######################
(9443, 5)
(3551, 5) eia8602022
(14239, 5)
(2227, 5) eia86020

In [140]:
# raw_plant_df = raw_plant_df[list(eia860_common_cols)]
# raw_plant_df

# raw_generator_df = raw_generator_df[['plant_code'] + eia860_generator_cols]
# raw_generator_df = raw_generator_df[pd.isna(raw_generator_df['plant_code']) == False]
# raw_generator_df = raw_generator_df[raw_generator_df[ENERGY_SOURCE].isin(
#     REQUIRED_ENERGY_SOURCES)]
# raw_generator_df

# gen_grouped_df = raw_generator_df.groupby(
#     ['plant_code', 'operating_year', 'operating_month', 'energy_source_1']).sum()
# plant_grouped_df = raw_plant_df.groupby('plant_code').agg({
#     'sector': pd.Series.mode,
#     'plant_name': pd.Series.mode,
#     'state': pd.Series.mode,
#     'county': pd.Series.mode})

# gen_grouped_df.to_excel('eia860_gen_grouped_2022.xlsx')
# plant_grouped_df.to_excel('eia860_plant_grouped_2022.xlsx')

# gen_grouped_df


In [141]:
# ## ISSUE 1 as of 25th nov, 2023
# summer_nan_plant_codes = np.unique(
#     raw_generator_df[raw_generator_df[capacity_cols[1]] == '.']['PLANT_CODE'])
# winter_nan_plant_codes = np.unique(
#     raw_generator_df[raw_generator_df[capacity_cols[2]] == '.']['PLANT_CODE'])
# print(summer_nan_plant_codes)
# print(winter_nan_plant_codes)

# raw_generator_summer_df = raw_generator_df[raw_generator_df[capacity_cols[1]] != '.']
# summer_groupby_df = raw_generator_summer_df[['PLANT_CODE'] + [capacity_cols[1]]].groupby(
#     ['PLANT_CODE']).sum()

# raw_generator_winter_df = raw_generator_df[raw_generator_df[capacity_cols[2]] != '.']
# winter_groupby_df = raw_generator_winter_df[['PLANT_CODE'] + [capacity_cols[2]]].groupby(
#     ['PLANT_CODE']).sum()

# summer_groupby_df, winter_groupby_df

# print([plt for plt in summer_nan_plant_codes if plt not in summer_groupby_df.index])
# print([plt for plt in winter_nan_plant_codes if plt not in winter_groupby_df.index])

#### 2. Read EIA-923 files

In [72]:
EIA923_FILE = 'EIA923_Schedules_2_3_4_5'

eia923_folders = os.listdir(EIA923_DATA_FOLDER)
eia923_folders.remove('.DS_Store')
eia923_folders.sort()
eia923_folders

['f923_2011',
 'f923_2012',
 'f923_2013',
 'f923_2014',
 'f923_2015',
 'f923_2016',
 'f923_2017',
 'f923_2018',
 'f923_2019',
 'f923_2020',
 'f923_2021',
 'f923_2022']

In [73]:
required_files_dict = {}
for dir_ in eia923_folders:
    files_list = os.listdir(EIA923_DATA_FOLDER + dir_)
    print(dir_)
    required_file = [file for file in files_list if (
        EIA923_FILE in file) and ('~$' not in file)][0]
    required_files_dict[dir_] = required_file
#     print(required_files_dict)
#     print('###########################')
print(required_files_dict)

f923_2011
f923_2012
f923_2013
f923_2014
f923_2015
f923_2016
f923_2017
f923_2018
f923_2019
f923_2020
f923_2021
f923_2022
{'f923_2011': 'EIA923_Schedules_2_3_4_5_2011_Final_Revision.xlsx', 'f923_2012': 'EIA923_Schedules_2_3_4_5_M_12_2012_Final_Revision.xlsx', 'f923_2013': 'EIA923_Schedules_2_3_4_5_2013_Final_Revision.xlsx', 'f923_2014': 'EIA923_Schedules_2_3_4_5_M_12_2014_Final_Revision.xlsx', 'f923_2015': 'EIA923_Schedules_2_3_4_5_M_12_2015_Final_Revision.xlsx', 'f923_2016': 'EIA923_Schedules_2_3_4_5_M_12_2016_Final_Revision.xlsx', 'f923_2017': 'EIA923_Schedules_2_3_4_5_M_12_2017_Final_Revision.xlsx', 'f923_2018': 'EIA923_Schedules_2_3_4_5_M_12_2018_Final_Revision.xlsx', 'f923_2019': 'EIA923_Schedules_2_3_4_5_M_12_2019_Final_Revision.xlsx', 'f923_2020': 'EIA923_Schedules_2_3_4_5_M_12_2020_Final_Revision.xlsx', 'f923_2021': 'EIA923_Schedules_2_3_4_5_M_12_2021_Final_Revision.xlsx', 'f923_2022': 'EIA923_Schedules_2_3_4_5_M_12_2022_Final.xlsx'}


In [74]:
raw_eia923_df = pd.read_excel(
    EIA923_DATA_FOLDER + eia923_folders[-1] + '/' + required_files_dict[eia923_folders[-1]])
print(raw_eia923_df.shape, dir_, required_files_dict[dir_])
raw_eia923_df = raw_eia923_df.rename(columns=raw_eia923_df.iloc[4])
raw_eia923_df = raw_eia923_df.drop(raw_eia923_df.index[0:5])
print(raw_eia923_df.shape, eia923_folders[-1], required_files_dict[eia923_folders[-1]])

NETGEN_COLUMNS = [col for col in raw_eia923_df.columns if 'Netgen_' in col]
MONTH_NUMBERS_DICT = {key: i+1 for i, key in enumerate(NETGEN_COLUMNS)}
print(NETGEN_COLUMNS, MONTH_NUMBERS_DICT)

PLANT_ID = 'Plant Id'
PLANT_NAME = 'Plant Name'
STATE = 'State'
FUEL_SOURCE = 'Reported Fuel Type Code'
NETGEN_ANNUAL = 'Net Generation (Megawatthours)'
YEAR = 'YEAR'
print(NETGEN_COLUMNS)

(16458, 97) f923_2022 EIA923_Schedules_2_3_4_5_M_12_2022_Final.xlsx
(16453, 97) f923_2022 EIA923_Schedules_2_3_4_5_M_12_2022_Final.xlsx
['Netgen_Jan', 'Netgen_Feb', 'Netgen_Mar', 'Netgen_Apr', 'Netgen_May', 'Netgen_Jun', 'Netgen_Jul', 'Netgen_Aug', 'Netgen_Sep', 'Netgen_Oct', 'Netgen_Nov', 'Netgen_Dec'] {'Netgen_Jan': 1, 'Netgen_Feb': 2, 'Netgen_Mar': 3, 'Netgen_Apr': 4, 'Netgen_May': 5, 'Netgen_Jun': 6, 'Netgen_Jul': 7, 'Netgen_Aug': 8, 'Netgen_Sep': 9, 'Netgen_Oct': 10, 'Netgen_Nov': 11, 'Netgen_Dec': 12}
['Netgen_Jan', 'Netgen_Feb', 'Netgen_Mar', 'Netgen_Apr', 'Netgen_May', 'Netgen_Jun', 'Netgen_Jul', 'Netgen_Aug', 'Netgen_Sep', 'Netgen_Oct', 'Netgen_Nov', 'Netgen_Dec']


In [96]:
# ONLY NEED TO RUN ONCE - DONE
raw_eia923_df_dict = {}
for dir_ in required_files_dict:
    
    raw_eia923_df = pd.read_excel(
        EIA923_DATA_FOLDER + dir_ + '/' + required_files_dict[dir_])
    print(raw_eia923_df.shape, dir_, required_files_dict[dir_])
    raw_eia923_df = raw_eia923_df.rename(columns=raw_eia923_df.iloc[4])
    raw_eia923_df = raw_eia923_df.drop(raw_eia923_df.index[0:5])
    print(raw_eia923_df.shape, dir_, required_files_dict[dir_])
    raw_eia923_df = raw_eia923_df[raw_eia923_df['Reported Fuel Type Code'].isin(
        REQUIRED_ENERGY_SOURCES)]
    raw_eia923_df = raw_eia923_df[raw_eia923_df[PLANT_ID].isin(common_gen_grouped_list)]
    print(raw_eia923_df.shape, dir_, required_files_dict[dir_])
    raw_eia923_df.reset_index(inplace=True, drop=True)
    print(raw_eia923_df.shape)
    raw_eia923_df = raw_eia923_df.replace('.', 0)
    raw_eia923_df = raw_eia923_df[[PLANT_ID, NETGEN_ANNUAL] + NETGEN_COLUMNS].groupby(
        PLANT_ID).sum()
    raw_eia923_df.reset_index(inplace=True)
    print(raw_eia923_df.shape)
    raw_eia923_df_dict[dir_] = raw_eia923_df
    print('#########################')


(10466, 97) f923_2011 EIA923_Schedules_2_3_4_5_2011_Final_Revision.xlsx
(10461, 97) f923_2011 EIA923_Schedules_2_3_4_5_2011_Final_Revision.xlsx
(2225, 97) f923_2011 EIA923_Schedules_2_3_4_5_2011_Final_Revision.xlsx
(2225, 97)
(2202, 14)
#########################
(11009, 97) f923_2012 EIA923_Schedules_2_3_4_5_M_12_2012_Final_Revision.xlsx
(11004, 97) f923_2012 EIA923_Schedules_2_3_4_5_M_12_2012_Final_Revision.xlsx
(2235, 97) f923_2012 EIA923_Schedules_2_3_4_5_M_12_2012_Final_Revision.xlsx
(2235, 97)
(2208, 14)
#########################
(11471, 97) f923_2013 EIA923_Schedules_2_3_4_5_2013_Final_Revision.xlsx
(11466, 97) f923_2013 EIA923_Schedules_2_3_4_5_2013_Final_Revision.xlsx
(2226, 97) f923_2013 EIA923_Schedules_2_3_4_5_2013_Final_Revision.xlsx
(2226, 97)
(2201, 14)
#########################
(12177, 97) f923_2014 EIA923_Schedules_2_3_4_5_M_12_2014_Final_Revision.xlsx
(12172, 97) f923_2014 EIA923_Schedules_2_3_4_5_M_12_2014_Final_Revision.xlsx
(2222, 97) f923_2014 EIA923_Schedules_2_3_

In [97]:
print(len(common_gen_grouped_list))

2227


In [100]:
common_plants_eia923_list = list(raw_eia923_df_dict[eia923_folders[0]][PLANT_ID])
print(len(common_plants_eia923_list))

for name in eia923_folders[1:]:
    plant_code_list = list(raw_eia923_df_dict[name][PLANT_ID])
    common_plants_eia923_list = list(set(common_plants_eia923_list) & set(plant_code_list))
    print(len(common_plants_eia923_list), name)

print(len(common_plants_eia923_list), '#########')

print(set(common_gen_grouped_list) & set(
    common_plants_eia923_list) == set(common_gen_grouped_list))
print(set(common_gen_grouped_list) & set(
    common_plants_eia923_list) == set(common_plants_eia923_list))
# common_plants_eia923_list is a proper subset of common_gen_grouped_list
# use common_plants_eia923_list instead for [REF_LAT_LON] -> NARR climate subset

2202
2201 f923_2012
2189 f923_2013
2179 f923_2014
2174 f923_2015
2171 f923_2016
2166 f923_2017
2164 f923_2018
2160 f923_2019
2158 f923_2020
2135 f923_2021
2129 f923_2022
2129 #########
False
True


#### Get [REF_LAT_LON].xlsx

In [101]:
LATITUDE = 'latitude'
LONGITUDE = 'longitude'

In [103]:
final_lat_lon_df = raw_plant_df_dict[eia860_folders[-1]]
final_lat_lon_df = final_lat_lon_df[final_lat_lon_df['plant_code'].isin(
    common_plants_eia923_list)]
final_lat_lon_df = final_lat_lon_df[['plant_code', LATITUDE, LONGITUDE]]
final_lat_lon_df.reset_index(inplace=True, drop=True)
final_lat_lon_df

,plant_code,latitude,longitude
0,2,33.458665,-87.356823
1,4,32.583889,-86.283056
2,11,33.7845,-86.0524
3,12,33.2553,-87.4495
4,13,32.6189,-86.2548
...,...,...,...
2124,57939,32.866837,-111.771751
2125,57947,32.55434,-107.50929
2126,57948,39.380565,-75.188312
2127,57956,35.325278,-98.071389


In [104]:
# DONE WITH THIS
# final_lat_lon_df.to_excel('REF_LAT_LON.xlsx', index=False)

#### Combine EIA-860 and EIA-923 files

In [110]:
# raw_eia923_df.to_excel('raw_eia923_df_2022.xlsx', index=False)

In [112]:
# i am here

In [117]:
[int(k.split('f923_')[1]) for k in eia923_folders]

[2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [123]:
eia923_master_monthly_df = df({PLANT_CODE: [], MONTH: [], YEAR: [], GENERATION: []})

x = {PLANT_CODE: [2], MONTH: [2], YEAR: [2022], GENERATION: [1.4]}

eia923_master_monthly_df = pd.concat([eia923_master_monthly_df, df(x)])
eia923_master_monthly_df = pd.concat([eia923_master_monthly_df, df(x)])
eia923_master_monthly_df = pd.concat([eia923_master_monthly_df, df(x)])
eia923_master_monthly_df = pd.concat([eia923_master_monthly_df, df(x)])
eia923_master_monthly_df = pd.concat([eia923_master_monthly_df, df(x)])
eia923_master_monthly_df.reset_index(inplace=True)
eia923_master_monthly_df

,index,plant_code,month,year,net_gen
0,0,2.0,2.0,2022.0,1.4
1,0,2.0,2.0,2022.0,1.4
2,0,2.0,2.0,2022.0,1.4
3,0,2.0,2.0,2022.0,1.4
4,0,2.0,2.0,2022.0,1.4


In [147]:
PLANT_CODE = 'plant_code'
MONTH = 'month'
YEAR = 'year'
GENERATION = 'net_gen_mwh'
CAPACITY = 'nameplate_capacity_mw'


In [129]:
eia923_master_monthly_df = df({PLANT_CODE: [], MONTH: [], YEAR: [], GENERATION: []})
print(eia923_master_monthly_df.shape)
for dir_ in raw_eia923_df_dict:
    raw_eia923_df = raw_eia923_df_dict[dir_]
    print(raw_eia923_df.shape, '###########', dir_)
    for ind, row in raw_eia923_df.iterrows():
        for month_col in NETGEN_COLUMNS:
            append_dict = {PLANT_CODE: [row[PLANT_ID]],
                           MONTH: [MONTH_NUMBERS_DICT[month_col]],
                           YEAR: [int(dir_.split('f923_')[1])],
                           GENERATION: [row[month_col]]}
            eia923_master_monthly_df = pd.concat([eia923_master_monthly_df, df(append_dict)])
#             eia923_master_monthly_df = eia923_master_monthly_df.append(
#                 append_dict, ignore_index=True)
eia923_master_monthly_df.reset_index(inplace=True, drop=True)
print(eia923_master_monthly_df.shape)

eia923_master_monthly_df.to_excel(
    FILTER_DATA_STORE_PATH + 'eia923_master_df.xlsx', index=False)

(0, 4)
(2202, 14) ########### f923_2011
(2208, 14) ########### f923_2012
(2201, 14) ########### f923_2013
(2200, 14) ########### f923_2014
(2205, 14) ########### f923_2015
(2207, 14) ########### f923_2016
(2203, 14) ########### f923_2017
(2206, 14) ########### f923_2018
(2203, 14) ########### f923_2019
(2205, 14) ########### f923_2020
(2186, 14) ########### f923_2021
(2183, 14) ########### f923_2022
(316908, 4)


In [160]:
# new_plants_list = np.unique(list(
#     gen_grouped_df[gen_grouped_df[OPERATING_YEAR] > 2011][PLANT_CODE]))
# old_plants_list = np.unique(list(
#     gen_grouped_df[gen_grouped_df[OPERATING_YEAR] <= 2011][PLANT_CODE]))
# print(len(old_plants_list), len(new_plants_list), len(old_plants_list) + len(new_plants_list))

# common_old_new_plants_list = [plt for plt in new_plants_list if plt in old_plants_list]
# print(len(common_old_new_plants_list))

In [158]:
eia860_master_cap_df = df({PLANT_CODE: [], MONTH: [], YEAR: [],
                           ENERGY_SOURCE: [], CAPACITY: []})

for dir_ in final_gen_grouped_df_dict:
    gen_grouped_df = final_gen_grouped_df_dict[dir_]
    print(gen_grouped_df.shape, '###########', dir_)
    for ind, row in gen_grouped_df.iterrows():
        
        if not (((eia860_master_cap_df[PLANT_CODE] == row[PLANT_CODE]) & 
                 (eia860_master_cap_df[MONTH] == row[OPERATING_MONTH]) & 
                 (eia860_master_cap_df[YEAR] == row[OPERATING_YEAR])).any()):
            append_dict = {PLANT_CODE: [row[PLANT_CODE]],
                           MONTH: [row[OPERATING_MONTH]],
                           YEAR: [row[OPERATING_YEAR]],
                           ENERGY_SOURCE: [row[ENERGY_SOURCE]],
                           CAPACITY: [row[NAMEPLATE_CAPACITY]]}
            eia860_master_cap_df = pd.concat([eia860_master_cap_df, df(append_dict)])
#             eia923_master_monthly_df = eia923_master_monthly_df.append(
#                 append_dict, ignore_index=True)
eia860_master_cap_df.reset_index(inplace=True, drop=True)
print(eia860_master_cap_df.shape)

eia860_master_cap_df = eia860_master_cap_df[eia860_master_cap_df[PLANT_CODE].isin(
    common_plants_eia923_list)]
eia860_master_cap_df.to_excel(
    FILTER_DATA_STORE_PATH + 'eia860_master_cap_df.xlsx', index=False)    
    
    

(3494, 5) ########### eia8602011
(3521, 5) ########### eia8602012
(3530, 5) ########### eia8602013
(3532, 5) ########### eia8602014
(3539, 5) ########### eia8602015
(3543, 5) ########### eia8602016
(3542, 5) ########### eia8602017
(3547, 5) ########### eia8602018
(3543, 5) ########### eia8602019
(3543, 5) ########### eia8602020
(3545, 5) ########### eia8602021
(3551, 5) ########### eia8602022
(3646, 5)


In [ ]:
def get_cap_col(r):
    
    return r

eia923_master_monthly_df = eia923_master_monthly_df.apply(get_cap_col, axis=1)

In [161]:
eia860_master_cap_df

,plant_code,month,year,energy_source_1,nameplate_capacity_mw
0,2.0,7.0,1963.0,WAT,45.0
1,4.0,7.0,1967.0,WAT,75.0
2,4.0,8.0,1967.0,WAT,150.0
3,11.0,6.0,1966.0,WAT,72.9
4,12.0,8.0,1968.0,WAT,40.0
...,...,...,...,...,...
3640,6422.0,3.0,2022.0,WAT,4.1
3642,50654.0,7.0,2022.0,SUN,2.0
3643,54111.0,11.0,2022.0,GEO,42.5
3644,56336.0,2.0,2022.0,WND,145.0


In [162]:
eia923_master_monthly_df

,plant_code,month,year,net_gen
0,2.0,1.0,2011.0,11312.437
1,2.0,2.0,2011.0,9090.908
2,2.0,3.0,2011.0,22446.993
3,2.0,4.0,2011.0,20390.027
4,2.0,5.0,2011.0,7243.627
...,...,...,...,...
316903,58080.0,8.0,2022.0,23394.840
316904,58080.0,9.0,2022.0,21587.098
316905,58080.0,10.0,2022.0,28148.298
316906,58080.0,11.0,2022.0,34128.740
